# ca1 : الگوریتم ژنتیک
#### mahdi vajhi
#### 810101558
<p dir=rtl style="text-align:right">
هدف از این پروژه حل مسئله 
Problem Knapsack
با استفاده از الگوریتم ژنتیک است. ما در این مسئله باید حالت بهینه از غذا ها را که شرایط ورودی را داشته باشد پیدا کنیم.
برای این کار ما ابتدا باید ساختار کروموزوم های خود را تعریف کنیم که دارای قابلیت تلفیق با بقیه و جهش را داشته باشد. سپس باید تعدادی کروموزوم اولیه تولید کنیم و با پیاده سازی یک تابع فیتنس برای آنها میزان خوب بودن آنها را بسنجیم این تابع باید به شرایط مسئله حساس باشد و همچنین میزان خوب بودن ژن ها را نشان دهد. سپس باید تابعی برای تولید نسل بعدی نوشت که ابتدا کروموزوم ها را جفت کند سپس آنها را تلفیق کند و در نهایت آنها را جهش دهد. در آخر هم باید نتایج را بررسی کرد و جواب به دست آمده را بررسی کنیم.
</p>

## initial block

<p dir=rtl>
ثابت های زیر مقدار اولیه هستند و در آخر مقادیر متفاوت نیز آزمایش شده
</p>

In [287]:
import random
import pandas as pd
import numpy as np
import copy
import bisect 


GEN_ACTIVE_RATE = 0.5
WIGHT_MUTATION_RATE = 0.05
MIN_WEIGHT = 0.5
IS_WIGHT_MUTATION_IF_DEACTIVATE = True
FIRST_GENERATION_COUNT = 10
CROSSOVER_CHANCE = 1/3
SWAP_CHANCE_IN_CROSSOVER = 1/5
POSITIVE_VALUE_RATE = 0.1
POSITIVE_WEIGHT_RATE = 0
POSITIVE_COUNT_RATE = 0
FITNESS_VALUE_RATE = 1
FITNESS_WEIGHT_RATE = 1
FITNESS_COUNT_RATE = 1
HAVE_CROSSOVER = True
HAVE_MUTATION = True
GENERATION_STEP = 1000
INCREASE_GENERATION_RATE = 1
OLD_CHORMO_COUNT = 5

## تعریف کروموزوم و ژن ها
<p dir=rtl>
در این مسئله با توجه به این که طول کروموزوم باید ثابت باشد و همچنین تعداد غذا ها ثابت است می توانیم کروموزوم را فهرست تمام غذا ها بگذاریم که با یک متغییر مشخص شود که آیا آن غذا را می بریم یا خیر و اگر می بریم چه مقدار از آن را می بریم. پس در این مسئله ژن ها همان غذا ها هستند که می تواند فعال یا غیر فعال باشند.
</p>

### ژن ها
<p dir=rtl>
کلاس ژن در واقع نماینده یک غذا است که در آن نام، حداکثر مقدار موجود، فعال بودن آن، چگالی ارزش آن و وزن الان آن است.
</p>
<p dir=rtl>
در تابع جهش ژن با احتمالی که تعیین شده می تواند فعال یا غیر فعال شود
همچنین وزن آن با یک توزیع نرمال حول مقدار قبلی خود تغییر می کند. توزیع به گونه ای تنظیم شده که انحراف معیار به اندازه یک ثابت بین ۰ تا ۱ در مقدار وزن باشد که تغییر معقول باشد. همچنین شرطی گذاشته شده که آیا درموقع غیر فعال بودن ژن وزن آن جهش کند یا خیر
</p>


In [288]:
class Gen:
    def __init__(self, name:str, max_weight:float, value_per_weight:float
                 , weight:float=0, is_active:bool=False):
        self.name = name
        self.max_weight = max_weight
        self.vpw = value_per_weight
        self.is_active = is_active
        self.weight = weight

    def set_random_gen(self):
        self.set_random_gen_status()
        self.set_random_wight()
    def set_random_gen_status(self):
        self.is_active = random.choice([True, False])
    def set_random_wight(self):
        self.weight = round(random.uniform(MIN_WEIGHT, self.max_weight),1)

    def mutation(self):
        if IS_WIGHT_MUTATION_IF_DEACTIVATE or self.is_active:
            self.weight_mutation()
        self.active_mutation()
    def active_mutation(self):
        self.is_active = self.is_active if random.random() < GEN_ACTIVE_RATE\
                    else not self.is_active
    def weight_mutation(self):
        std = self.weight * WIGHT_MUTATION_RATE
        self.weight = np.random.normal(self.weight, std, 1)[0]
        # fix number in range and round
        self.weight = round((max(MIN_WEIGHT, min(self.max_weight, self.weight))),1)
        
    # Getters
    def get_weight(self):
        return self.max_weight if self.is_active else 0
    def get_value(self):
        return self.max_weight * self.vpw if self.is_active else 0
    def get_name(self):
        return self.name
    def get_is_active(self):
        return self.is_active
    def __str__(self):
        return "------------\n" + f"name: {self.name}\nstatus: {self.is_active}\n" +\
        f"weight: {self.weight}\nmax weight: {self.max_weight}\nvpw: {self.vpw:.2f}"
    def __repr__(self):
        return self.__str__()

### کروموزوم ها
<p dir=rtl>
کلاس کروموزوم صرفا اجماع ژن ها است و توابع آن صرفا روی ژن ها حلقه می زند.
</p>
<p dir=rtl>
فقط تابع تلفیق ژن ها اضافه شده. برای این کار اپراتور جمع بازنویسی شده و با یک احتمالی کروموزوم ها را تلفیق می کند ویا همان ها را بر می گرداند. برای تقاطع دادن آن ها از تقاطع با احتمال یکسان استفاده شده چون توالی غذا ها مفهوم خاصی ندارد و دلیلی ندارد به خاطر متوالی بودن دو غذا احتمال تقاطع آنها وابسته به هم باشد. در آخر با یک swap کردن دو کروموزوم جدید را ساختیم.
</p>

In [289]:
class Chromosome:
    def __init__(self, gens:list[Gen]):
        self.gens = gens

    def mutation(self):
        for i in self.gens:
            i.mutation()
    
    # Uniform crossover
    def __add__(self, pair):
        if random.choices([True, False],
                          [CROSSOVER_CHANCE,1 - CROSSOVER_CHANCE])[0]:
            return self.crossover(pair)
        return self, pair
    
    def crossover(self, pair):
        swap_gen_status = self.get_swap_gen_status()
        new_chromosomes = [[0] * len(self.gens)] * 2
        for i in range(len(self.gens)):
            if swap_gen_status[i]:
                new_chromosomes[0][i], new_chromosomes[1][i] =\
                pair.gens[i]         , self.gens[i]
            else:
                new_chromosomes[0][i], new_chromosomes[1][i] =\
                self.gens[i]         , pair.gens[i]
        return Chromosome(copy.deepcopy(new_chromosomes[0])),\
               Chromosome(copy.deepcopy(new_chromosomes[1]))
    
    def get_swap_gen_status(self):
        return random.choices([True, False],
            [SWAP_CHANCE_IN_CROSSOVER, 1 - SWAP_CHANCE_IN_CROSSOVER],
            k=len(self.gens))
        
    # Getters
    def get_weight(self):
        return sum([i.get_weight() for i in self.gens])
    def get_value(self):
        return sum([i.get_value() for i in self.gens])
    def get_vpw(self):
        return self.get_value() / self.get_weight()
    def get_name(self):
        return [i.get_name() for i in self.gens]
    def get_active_count(self):
        return sum([i.get_is_active() for i in self.gens])
    
    def __str__(self):
        return "__________\n"+\
                f"active_count: {self.get_active_count()}\n"+\
                f"weight: {self.get_weight()}\n"+\
                f"value: {self.get_value()}\n"+\
                f"vpw: {self.get_vpw():.2f}"
    def __repr__(self):
        return self.__str__()
    
    def __lt__(self, x):
        return calculate_fitness(self, conditions) > calculate_fitness(x,conditions)
    def __gt__(self, x):
        return calculate_fitness(self, conditions) < calculate_fitness(x,conditions)

## تولید جمعیت اولیه
<p dir=rtl>
در این قسمت باید جامعه اولیه را تولید کنیم که شامل مراجل زیر است:
</p>
<ol dir=rtl>
    <li> خواندن داده ها </li>
    <li> تولید ژن خام از غذا ها </li>
    <li> تولید کرموزوم ها با ژن های خام و مقدار دهی شانسی وزن آنها و وضعیت شانسی فعال بودن ژن ها </li>
</ol>

### read data
<p dir=rtl style="text-align:right">
در این قسمت فایل
snacks.csv
را لود می کنیم که داده های غذا های مربوط به پروژه در آن قرار دارد
</p>

In [290]:
snacks = pd.read_csv('./snacks.csv')
snacks

,Snack,Available Weight,Value
0,MazMaz,10,10
1,Doogh-e-Abali,15,10
2,Nani,5,5
3,Jooj,7,15
4,Hot-Dog,20,15
5,Chips,8,6
6,Nooshaba,12,8
7,Shokolat,6,7
8,Chocoroll,9,12
9,Cookies,11,11


### ساخت ژن های خام
<p dir=rtl>
در این قسمت داده های جدول را باید به ژن ها تبدیل کنیم اما مقدار وزن و فعال بودن آن در کروموزوم تعیین می شود
</p>

In [291]:
row_gens = list[Gen]()
for i in snacks.iterrows():
    name, max_weight, value = i[1]
    row_gens.append(Gen(name, max_weight, value/max_weight))

In [292]:
for i in row_gens[:2]:
    print(i)

------------
name: MazMaz
status: False
weight: 0
max weight: 10
vpw: 1.00
------------
name: Doogh-e-Abali
status: False
weight: 0
max weight: 15
vpw: 0.67


### ساخت کروموزوم های اولیه
<p dir=rtl>
در این قسمت ژن ها را برای هر کروموزوم مقدار شانسی می دهیم و کروموزوم های اولیه را می سازیم. برای تعداد جمعیت پارامتر هایی تعریف می کنیم.
</p>

<p dir=rtl>
تابع زیر با استفاده از ژن های خام یک کروموزوم رندوم می سازد
</p>

In [293]:
def create_random_chromosome() -> Chromosome:
    gens = list[Gen]()
    for i in row_gens:
        new_gen = copy.deepcopy(i)
        new_gen.set_random_gen()
        gens.append(new_gen)
    return Chromosome(gens)

<p dir=rtl>
تابع زیر با کمک تابع قبل کروموزوم های اولیه را می سازد
</p>

In [294]:
def create_first_generation() -> list[Chromosome]:
    first_chromosomes = list[Chromosome]()
    for _ in range(FIRST_GENERATION_COUNT):
        first_chromosomes.append(create_random_chromosome())
    return first_chromosomes

<p dir=rtl>
کارکرد آن را بررسی می کنیم
</p>

In [295]:
test = create_first_generation()
for i in test[:2]:
    print(r'###############')
    print(i)
    print(r'__________')
    for j in i.gens[:2]:
        print(j)

###############
__________
active_count: 12
weight: 134
value: 120.0
vpw: 0.90
__________
------------
name: MazMaz
status: False
weight: 1.0
max weight: 10
vpw: 1.00
------------
name: Doogh-e-Abali
status: True
weight: 10.4
max weight: 15
vpw: 0.67
###############
__________
active_count: 9
weight: 91
value: 87.0
vpw: 0.96
__________
------------
name: MazMaz
status: False
weight: 2.4
max weight: 10
vpw: 1.00
------------
name: Doogh-e-Abali
status: False
weight: 5.5
max weight: 15
vpw: 0.67


## ساخت تابع فیتنس
<p dir=rtl>
برای طراحی تابع فیتنس باید موارد زیر را در نظر بگیریم:
</p>
<ul dir=rtl>
    <li> بیشتر بودن ارزش غذایی از ورودی مسئله </li>
    <li> کمتر بودن وزن از ورودی مسئله </li>
    <li> قرار گرفتن تعداد غذا ها در بازه ورودی مسئله </li>
</ul>
<p dir=rtl>
چون حداکثر وزن هر خوراکی در ژن ها اعمال شده ژن ها از آن عبور نمی کنند بنابراین نیازی به لحاظ کردن آن نیست.
</p>
<p dir=rtl>
موارد زیر را می توان نکات مثبت برای کرموزوم در نظر گرفت:
</p>
<ul dir=rtl>
    <li> بالا بودن vpw </li>
    <li> بالا بودن ارزش </li>
</ul>

In [296]:
def calculate_value_fitness(value:float, condition:float):
    diff = value - condition
    if diff < 0:
        return diff
    # TODO
    return diff * POSITIVE_VALUE_RATE

def calculate_weight_fitness(weight:float, condition:float):
    diff = condition - weight
    if diff < 0:
        return diff
    # TODO
    return diff * POSITIVE_WEIGHT_RATE

def calculate_count_fitness(count:int, condition:list[int]):
    if count < condition[0]:
        return count - condition[0]
    if count > condition[1]:
        return condition[1] - count
    else:
        return POSITIVE_COUNT_RATE

In [297]:
def calculate_fitness(chromosome:Chromosome, conditions:dict):
    value_point =   calculate_value_fitness(chromosome.get_value(), conditions["value"])
    weight_point =  calculate_weight_fitness(chromosome.get_weight(), conditions["weight"])
    count_point =   calculate_count_fitness(chromosome.get_active_count(), conditions["count"])
    return  value_point  *  FITNESS_VALUE_RATE +\
            weight_point *  FITNESS_WEIGHT_RATE +\
            count_point  *  FITNESS_COUNT_RATE 

## تولید نسل بعدی
<p dir=rtl>
تابع جهش و تلفیق در کلاس ها نوشته شده در این بخش با استفاده از آنها و تابع فیتنس نسل بعد را تولید می کنیم.
</p>

In [298]:
def calculate_generation_fitness(generation:list[Chromosome], conditions):
    fitness = list[float]()
    for i in generation:
        fitness.append(calculate_fitness(i, conditions))
    return fitness

<p dir=rtl>
تابع زیر جهت تبدیل فیتنس به احتمال انتخاب است. به این صورت که کوچکترین عدد را اگر از ۱ کوچک تر است همه اعداد را طوری جمع می زند که آن عدد ۱ شود و سپس احتمال را برابر نسبت فیتنس به جمع کل فیتنس ها قرار می دهد
</p>

In [299]:
def convert_fitness_to_probability_of_choice(generation_fitness:list[float]):
    minimum = min(generation_fitness)
    if minimum <= 0:
        generation_fitness = [i + abs(minimum) + 1 for i in generation_fitness]
    sum_all = sum(generation_fitness)
    return [i/sum_all for i in generation_fitness]

<p dir=rtl>
مراحل تابع تولید نسل به صورت زیر است:
</p>
<ol dir=rtl>
    <li> تبدیل فیتنس ها به احتمال </li>
    <li> جفت کردن کروموزوم ها با در نظر گرفتن احتمال </li>
    <li> تولید کروموزوم های جدید با تلفیق جفت ها </li>
    <li> جهش کروموزوم ها </li>
    <li> افزودن کروموزوم ها به نسل جدید </li>
</ol>

In [300]:
def pair_chromosomes(generation, probability_of_choice) -> tuple[Chromosome, Chromosome]:
    return np.random.choice(generation, 2, p=probability_of_choice, replace=False)
def create_children(pair:tuple[Chromosome, Chromosome]) -> tuple[Chromosome, Chromosome]:
        children = tuple[Chromosome, Chromosome]()
        
        if HAVE_CROSSOVER:
            children = pair[0] + pair[1]
        else:
            children = copy.deepcopy(pair)
        
        if HAVE_MUTATION:
            for i in children:
                i.mutation()
        
        return children

In [301]:
def create_next_generation(generation:list[Chromosome], generation_fitness:list[float],
                           size:int, conditions) -> list[Chromosome]:
    probability_of_choice = convert_fitness_to_probability_of_choice(generation_fitness)
    next_generation = list[Chromosome]()
    for _ in range(round(size/2)):
        pair = pair_chromosomes(generation, probability_of_choice)
        
        children = create_children(pair)

        next_generation.extend(children)
    
    return next_generation

## ایجاد الگوریتم ژنتیک
<p dir=rtl>
در آخر باید الگوریتم ژنتیک را پیاده کنیم که مراحل زیر را دارد:
</p>
<ol dir=rtl>
    <li> تولید نسل اول </li>
    <li>  تولید نسل های بعدی و محاسبه فیتنس آنها </li>
    <li> نمایش بهترین های نسل آخر به عنوان جواب </li>
</ol>

In [302]:
def check_chromosome(chromosome:Chromosome, conditions):
    return calculate_value_fitness(chromosome.get_value(), conditions["value"]) >= 0 and\
           calculate_weight_fitness(chromosome.get_weight(), conditions["weight"]) >= 0 and\
           calculate_count_fitness(chromosome.get_active_count(), conditions["count"]) >= 0
def get_generation_size(step:int):
    return (step + 1) * INCREASE_GENERATION_RATE * FIRST_GENERATION_COUNT
def find_best_chromosome(generation:list[Chromosome], generation_fitness:list[float]):
    return generation[np.argmax(generation_fitness)]

In [303]:
def solve_with_genetic(conditions):
    best_list = list()
    generation = create_first_generation()
    generation_fitness = calculate_generation_fitness(generation, conditions)
    for i in range(GENERATION_STEP):
        generation = create_next_generation(generation, generation_fitness
                                            , get_generation_size(i), conditions)
        generation_fitness = calculate_generation_fitness(generation, conditions)
        best_chromosome = find_best_chromosome(generation, generation_fitness)
        best_list.append(best_chromosome)
        if i%10 == 0:
            print(calculate_fitness(best_chromosome, conditions), np.average([calculate_fitness(i, conditions) for i in generation]))
        if check_chromosome(best_chromosome, conditions):
            print(calculate_fitness(best_chromosome, conditions), np.average([calculate_fitness(i, conditions) for i in generation]))
            return best_list
    return best_list

## بررسی نتایج

<p dir=rtl>
ابتدا با یک تنظیم اولیه آزمایش می کنیم و می بینیم که میانگین تغییری نمی کند ولی به جواب می رسیم
</p>

In [305]:
conditions = {"weight": 10,
              "value":12,
              "count": [2,4]}

GEN_ACTIVE_RATE = 0.05
WIGHT_MUTATION_RATE = 0.05
MIN_WEIGHT = 0.5
IS_WIGHT_MUTATION_IF_DEACTIVATE = True
FIRST_GENERATION_COUNT = 20
CROSSOVER_CHANCE = 1/2
SWAP_CHANCE_IN_CROSSOVER = 1/5
POSITIVE_VALUE_RATE = 0.1
POSITIVE_WEIGHT_RATE = 0
POSITIVE_COUNT_RATE = 0
FITNESS_VALUE_RATE = 1
FITNESS_WEIGHT_RATE = 1
FITNESS_COUNT_RATE = 1
HAVE_CROSSOVER = True
HAVE_MUTATION = True
GENERATION_STEP = 200
INCREASE_GENERATION_RATE = 1

ans = solve_with_genetic(conditions)
generation_fitness = calculate_generation_fitness(ans, conditions)
best_chromosome = find_best_chromosome(ans, generation_fitness)
print(best_chromosome)
print(f"step: {len(ans)}")
print(f"size: {FIRST_GENERATION_COUNT}")

-31.3 -75.825
-23.3 -81.28499999999998


-19.2 -80.5995238095238
-20.3 -80.87903225806451
-16.2 -80.55768292682927
-17.7 -79.86872549019607
-13.5 -81.56122950819672
-18.0 -80.41267605633803
-8.7 -80.30475308641975
-8.2 -81.03357142857143
-10.8 -80.71633663366336
-14.8 -80.12423423423422
-8.9 -80.58388429752065
-5.3 -80.76492366412214
0.0 -81.1152962962963
__________
active_count: 2
weight: 8
value: 12.0
vpw: 1.50
step: 135
size: 20


<p dir=rtl>
به نظر اگر تعدادی از بهترین های قبلی را به نسل جدید تزریق کنیم نتیجه بهتر شود اما این مورد هم خیلی موثر نیست
</p>

In [271]:
def solve_with_genetic(conditions):
    best_list = list[Chromosome]()
    generation = create_first_generation()
    generation_fitness = calculate_generation_fitness(generation, conditions)
    for i in range(GENERATION_STEP):
        generation = create_next_generation(generation, generation_fitness
                                            , get_generation_size(i), conditions)
        
        generation.extend(best_list[:min(len(best_list), OLD_CHORMO_COUNT)])
        generation_fitness = calculate_generation_fitness(generation, conditions)
        best_chromosome = find_best_chromosome(generation, generation_fitness)
        best_list.append(best_chromosome)
        if i%10 == 0:
            print(calculate_fitness(best_chromosome, conditions), np.average([calculate_fitness(i, conditions) for i in generation]))
        if check_chromosome(best_chromosome, conditions):
            print(calculate_fitness(best_chromosome, conditions), np.average([calculate_fitness(i, conditions) for i in generation]))
            return best_list
    return best_list

In [272]:
conditions = {"weight": 10,
              "value":12,
              "count": [2,4]}

GEN_ACTIVE_RATE = 0.05
WIGHT_MUTATION_RATE = 0.05
MIN_WEIGHT = 0.5
IS_WIGHT_MUTATION_IF_DEACTIVATE = True
FIRST_GENERATION_COUNT = 15
CROSSOVER_CHANCE = 1/2
SWAP_CHANCE_IN_CROSSOVER = 1/5
POSITIVE_VALUE_RATE = 0.1
POSITIVE_WEIGHT_RATE = 0
POSITIVE_COUNT_RATE = 0
FITNESS_VALUE_RATE = 1
FITNESS_WEIGHT_RATE = 1
FITNESS_COUNT_RATE = 1
HAVE_CROSSOVER = True
HAVE_MUTATION = True
GENERATION_STEP = 200
INCREASE_GENERATION_RATE = 1
OLD_CHORMO_COUNT = 5

ans = solve_with_genetic(conditions)
generation_fitness = calculate_generation_fitness(ans, conditions)
best_chromosome = find_best_chromosome(ans, generation_fitness)
print(best_chromosome)
print(f"step: {len(ans)}")
print(f"size: {FIRST_GENERATION_COUNT}")

-13.2 -67.5875
-21.2 -85.01183431952663


-22.2 -81.91339563862928
-15.8 -79.68635394456288
-4.8 -78.19838969404186
-18.1 -80.23459037711312
-12.9 -80.00238870792616
-4.0 -78.94050514499531
-11.7 -79.66257166257166
-16.7 -79.89050401753104
-10.8 -79.77981591058514
-8.7 -81.10209706411025
-7.9 -79.65738605161998
-4.6 -79.54667343829357
0.0 -79.10438639724002
__________
active_count: 2
weight: 8
value: 12.0
vpw: 1.50
step: 135
size: 15


<p dir=rtl>
شاید با سنگین تر کردن تابع فیتنس جواب ها بهتر شوند
</p>

In [284]:
def calculate_fitness(chromosome:Chromosome, conditions:dict):
    value_point =   calculate_value_fitness(chromosome.get_value(), conditions["value"])
    weight_point =  calculate_weight_fitness(chromosome.get_weight(), conditions["weight"])
    count_point =   calculate_count_fitness(chromosome.get_active_count(), conditions["count"])
    output = 0
    for i in [value_point, weight_point, count_point]:
        if i < -1:
            output -= i**2
        else:
            output += i
    return output

In [285]:
def solve_with_genetic(conditions):
    best_list = list()
    generation = create_first_generation()
    generation_fitness = calculate_generation_fitness(generation, conditions)
    for i in range(GENERATION_STEP):
        generation = create_next_generation(generation, generation_fitness
                                            , get_generation_size(i), conditions)
        generation_fitness = calculate_generation_fitness(generation, conditions)
        best_chromosome = find_best_chromosome(generation, generation_fitness)
        best_list.append(best_chromosome)
        if i%10 == 0:
            print(calculate_fitness(best_chromosome, conditions), np.average([calculate_fitness(i, conditions) for i in generation]))
        if check_chromosome(best_chromosome, conditions):
            print(calculate_fitness(best_chromosome, conditions), np.average([calculate_fitness(i, conditions) for i in generation]))
            return best_list
    return best_list

In [286]:
conditions = {"weight": 10,
              "value":12,
              "count": [2,4]}

GEN_ACTIVE_RATE = 0.05
WIGHT_MUTATION_RATE = 0.05
MIN_WEIGHT = 0.5
IS_WIGHT_MUTATION_IF_DEACTIVATE = True
FIRST_GENERATION_COUNT = 20
CROSSOVER_CHANCE = 1/2
SWAP_CHANCE_IN_CROSSOVER = 1/5
POSITIVE_VALUE_RATE = 0.1
POSITIVE_WEIGHT_RATE = 0
POSITIVE_COUNT_RATE = 0
FITNESS_VALUE_RATE = 1
FITNESS_WEIGHT_RATE = 1
FITNESS_COUNT_RATE = 1
HAVE_CROSSOVER = True
HAVE_MUTATION = True
GENERATION_STEP = 200
INCREASE_GENERATION_RATE = 1

ans = solve_with_genetic(conditions)
generation_fitness = calculate_generation_fitness(ans, conditions)
best_chromosome = find_best_chromosome(ans, generation_fitness)
print(best_chromosome)
print(f"step: {len(ans)}")
print(f"size: {FIRST_GENERATION_COUNT}")

-1761.7 -8433.61
-1939.2 -7894.2072727272725
-287.0 -7722.475714285714


-725.7 -7240.724193548387
-287.7 -7244.392926829269
-623.3 -7073.381764705883
-167.0 -7484.453278688525
-168.0 -7137.9850704225355
-195.3 -7378.312345679013
-253.5 -7250.674120879121
-287.3 -7212.41896039604
-622.0 -7440.978243243243
-224.2 -7288.810289256197
-79.5 -7257.928091603054
-120.0 -7269.672517730496
-194.4 -7278.369735099338


KeyboardInterrupt: 